# Instalação
#INSTALAÇÃO DAS LIBS
- !pip install yfinance==0.2.41
- !pip install crewai==0.28.8
- !pip install 'crewai[tools]'
- !pip install langchain==0.1.20
- !pip install langchain-openai==0.1.7
- !pip install langchain-community==0.0.38
- !pip install duckduckgo-search==5.3.0

In [105]:
import json
import os

from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import display, Markdown

In [83]:
# CRIANDO YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock


yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func=lambda ticket: fetch_stock_price(ticket)
)

In [84]:
print(fetch_stock_price('AAPL'))

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043777   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546387   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date         

In [85]:
import os

# Importando OpenAI LLM - GPT
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o")

In [86]:
stockPriceAnalyst = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock price and analyses trends",
    backstory="""You're highly experienced in analyzing the price of an specific stock
    and make predictions about its future price.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[yahoo_finance_tool],
    allow_delegation=False
)

In [87]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output="""" Specify the current trend stock price - up, down or sideways. 
    eg. stock= 'APPL, price UP'
""",
    agent=stockPriceAnalyst
)

In [88]:
# IMPORTANT A TOOL DE SEARCH
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [89]:
newsAnalyst = Agent(
    role="Stock News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with
    the news context. For each request stock asset, specify a numbet between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory="""You're highly experienced in analyzing the market trends and news and have tracked assest for more then 10 years.
    You're also master level analyts in the tradicional markets and have deep understanding of human psychology.
    You understand news, theirs tittles and information, but you look at those with a health dose of skepticism. 
    You consider also the source of the news articles. 
    """,
    verbose=True,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],
    allow_delegation=False
)

In [90]:
get_news = Task(
    description=f"""Take the stock and always include BTC to it (if not request).
    Use the search tool to search each one individually. 
    The current date is {datetime.now()}.
    Compose the results into a helpfull report""",
    expected_output=""""A summary of the overall market and one sentence summary for each request asset. 
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
""",
    agent=newsAnalyst
)

In [91]:
stockAnalystWrite = Agent(
    role="Senior Stock Analyts Writer",
    goal=""""Analyze the trends price and news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend. """,
    backstory="""You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
    and narratives that resonate with wider audiences. 
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. 
    You're able to hold multiple opinions when analyzing anything.
""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation=True
)

In [92]:
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary.
""",
    expected_output= """"An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:
    - 3 bullets executive summary 
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fead/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways.
    write in pt-br
""",
    agent = stockAnalystWrite,
    context = [getStockPrice, get_news]
)

In [93]:
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose = 2,
    process= Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15
)

2024-08-25 20:24:30,638 - 140700230648256 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [108]:
results = crew.kickoff(inputs={"ticket": "AMZN"})


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AMZN price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
To analyze the stock price history of AMZN and create a trend analysis, I need to gather information on its historical prices and any relevant news that might affect the trend. Then, I need a detailed analysis to determine whether the trend is up, down, or sideways.

First, I'll delegate the task of analyzing the stock price history to the Senior Stock Price Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior stock price Analyst", "task": "Analyze the stock price history of AMZN", "context": "We need to analyze the stock price history of AMZN and determine the trend (up, down, or sideways). Please provide detailed historical price data and any patterns or trends you notice."}


> Entering new CrewAgentExecutor chain...
Thought: I need to fetch the historical stock pr

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  140.619995  140.839996  138.419998  139.940002  139.940002   
2023-08-09  139.970001  140.320007  137.100006  137.850006  137.850006   
2023-08-10  139.070007  140.410004  137.490005  138.559998  138.559998   
2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
...                ...         ...         ...         ...         ...   
2024-08-01  189.289993  190.600006  181.869995  184.070007  184.070007   
2024-08-02  166.750000  168.770004  160.550003  167.899994  167.899994   
2024-08-05  154.210007  162.960007  151.610001  161.020004  161.020004   
2024-08-06  161.710007  165.080002  158.539993  161.929993  161.929993   
2024-08-07  166.550003  167.580002  161.429993  162.770004  162.770004   

               Volume  
Date      

[*********************100%%**********************]  1 of 1 completed

I need to gather the latest stock price data for AMZN and BTC to analyze their trends and significant price movements. 

Action: Yahoo Finance Tool
Action Input: {"ticket": "AMZN"}
 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  140.619995  140.839996  138.419998  139.940002  139.940002   
2023-08-09  139.970001  140.320007  137.100006  137.850006  137.850006   
2023-08-10  139.070007  140.410004  137.490005  138.559998  138.559998   
2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
...                ...         ...         ...         ...         ...   
2024-08-01  189.289993  190.600006  181.869995  184.070007  184.070007   
2024-08-02  166.750000  168.770004  160.550003  167.899994  167.899994   
2024-08-05  154.210007  162.960007  151.610001  161.020004  161.020004   
20

Thought: I have successfully gathered the stock price data for AMZN. Now, I need to fetch the stock price data for BTC.

Action: Yahoo Finance Tool
Action Input: {"ticket": "BTC-USD"}


> Entering new CrewAgentExecutor chain...
Thought: I need to gather the stock price data for AMZN and BTC over the past year to analyze trends and significant price movements.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AMZN"}


> Entering new CrewAgentExecutor chain...
Thought: I need to gather the stock price data for AMZN from the last year using the Yahoo Finance Tool.

Action: Yahoo Finance Tool
Action Input: {"ticker": "AMZN"}


> Entering new CrewAgentExecutor chain...
I need to gather the latest stock price data for AMZN and BTC from the last year to analyze their trends and any significant price movements.

Action: Yahoo Finance Tool
Action Input: {"ticker": "AMZN"}


> Entering new CrewAgentExecutor chain...
To gather the necessary stock price data for AMZN and BTC, I will first use 

In [96]:
list(results.keys())

['final_output', 'tasks_outputs']

In [101]:
len(results['tasks_outputs'])

3

In [109]:
Markdown(results['final_output'])

# 📈📊 Newsletter AMZN - Outubro 2023

## Resumo Executivo
- **Tendência de Preço das Ações:** Ações da AMZN em alta devido a relatórios robustos de ganhos e sentimento positivo do mercado.
- **Resumo das Notícias:** Desempenho financeiro forte e sentimento de mercado favorável impulsionam otimismo dos analistas.
- **Índice de Medo/Ganância:** Sentimento atual do mercado caracterizado pela ganância.

## Introdução

Nos últimos meses, as ações da Amazon (AMZN) têm mostrado um movimento ascendente notável, refletindo o otimismo dos investidores em torno dos seus relatórios de ganhos robustos e a confiança contínua em suas principais divisões: comércio eletrônico e computação em nuvem. Vamos explorar as razões por trás dessa tendência e o que esperar para o futuro próximo.

## Análise Detalhada

### Tendência de Preço das Ações

As ações da Amazon têm experimentado um aumento significativo, impulsionadas por relatórios de ganhos sólidos e um sentimento de mercado positivo. A empresa continua a expandir sua participação de mercado, especialmente em suas divisões de comércio eletrônico e computação em nuvem. Os analistas permanecem otimistas quanto ao crescimento contínuo da Amazon, prevendo que a tendência de alta deve continuar no curto prazo. Esse otimismo é sustentado pela performance financeira robusta e pela confiança dos investidores na capacidade da Amazon de inovar e liderar os mercados em que atua.

### Resumo das Notícias

Recentemente, a Amazon divulgou resultados financeiros que superaram as expectativas do mercado, impulsionando ainda mais o sentimento positivo em torno das suas ações. A empresa tem se beneficiado da crescente demanda por serviços de computação em nuvem e do aumento das vendas no comércio eletrônico. Essas áreas têm sido as principais fontes de receita e crescimento para a Amazon, e os analistas estão otimistas de que essa trajetória continuará.

### Índice de Medo/Ganância

O índice de medo/ganância, uma métrica que mede o sentimento do mercado, atualmente aponta para a ganância no caso das ações da Amazon. Esse indicador sugere que os investidores estão confiantes e dispostos a assumir mais riscos, refletindo um forte otimismo quanto ao futuro desempenho da empresa. Essa ganância pode, no entanto, levar a uma volatilidade maior no preço das ações, à medida que os investidores reagem rapidamente a novas informações.

## Conclusão

Em resumo, as ações da Amazon estão em uma clara trajetória ascendente, impulsionadas por fortes relatórios financeiros e otimismo do mercado. Com um índice de medo/ganância indicando ganância, o sentimento do mercado é de confiança e expectativa positiva. A previsão de tendência para o futuro próximo é de alta, com a Amazon continuando a expandir sua participação de mercado e a inovar em suas principais divisões.

Prepare-se para um futuro promissor com a Amazon, mas mantenha um olhar atento para possíveis volatilidades no mercado! 🚀

---

Espero que esta análise detalhada e informativa ajude a manter nossos stakeholders bem informados sobre o desempenho e as perspectivas futuras da Amazon.